# 函数加载

In [29]:
# ################################################
# 重采样
def resample_image(itk_image, out_spacing=[1, 1, 1]):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
 
    # 根据输出out_spacing设置新的size
    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / out_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / out_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / out_spacing[2]))
    ]
 
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
 
    resample.SetInterpolator(sitk.sitkBSpline)
 
    return resample.Execute(itk_image)

def resample_image_execute(input_image, output_image):
    
    # 使用sitk读取对应的数据
    original_img = sitk.ReadImage(input_image)
    print('原始图像的Spacing：', original_img.GetSpacing())
    print('原始图像的Size：', original_img.GetSize())

    # 对数据进行重采样
    resample_img = resample_image(original_img)
    print('经过resample之后图像的Spacing是：', resample_img.GetSpacing())
    print('经过resample之后图像的Size是：', resample_img.GetSize())
    
    sitk.WriteImage(resample_img, output_image)
    



# # ################################################
# # N4偏置场矫正
# # ################################################
# def N4BiasFieldCorrection(imagePath,saveMaskOrNot):
#     # 读nifty
#     input_image = sitk.ReadImage(imagePath)
#     # 设置蒙版，只对蒙版内的部分操作
#     mask_image = sitk.OtsuThreshold(input_image,0,1,200)
#     input_image = sitk.Cast(input_image, sitk.sitkFloat32)
#     # 主运行
#     corrector = sitk.N4BiasFieldCorrectionImageFilter()
#     output_image = corrector.Execute(input_image,mask_image)
#     output_image = sitk.Cast(output_image, sitk.sitkInt16)
    
#     # 设置导出文件名
#     output_path = ''
#     output_mask_path=''
#     if imagePath[-7:]=='.nii.gz':
#         output_path = imagePath[:-7]+'_N4.nii.gz'
#         output_mask_path = imagePath[:-7]+'_N4_mask.nii.gz'
#     elif imagePath[-4:]=='.nii':
#         output_path = imagePath[:-4]+'_N4.nii.gz'
#         output_mask_path = imagePath[:-4]+'_N4_mask.nii.gz'
#     else:
#         print('文件类型错误')
        
#     if saveMaskOrNot:
#         sitk.WriteImage(mask_image, output_mask_path)
#     sitk.WriteImage(output_image, output_path)
        
#     print(f'【N4 processed successfully】{imagePath}')
    
    
# ################################################
# 强度归一化
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

# 设置重要的元数据：origin、Direction、Spacing，并返回已经含它们了的SimpleITK格式数据，以便做保存NII等处理
def setMetaMessage(target, origin):
    target.SetDirection(origin.GetDirection())
    target.SetOrigin(origin.GetOrigin())
    target.SetSpacing(origin.GetSpacing())
    return target

# 主程序
def signalStrengthNormalization(imagePath, outputPath):
    # 查看原始的维度信息
    sitk_img = sitk.ReadImage(imagePath)     # <class 'SimpleITK.SimpleITK.Image'>
    img = sitk.GetArrayFromImage(sitk_img)       # <class 'numpy.ndarray'>
    # print('原始维度：%s \n体素大小：%s'%(str(img.shape),str(sitk_img.GetSpacing())))   # (92, 210, 190) (1.0, 1.0, 1.0)
    
    # 标准化
    img_std = np.resize(standardization(img.flatten()),img.shape)

    # 变成原始的nifty维度
    imgData_Processed = sitk.GetImageFromArray(img_std)
    imgData_Processed = setMetaMessage(imgData_Processed, sitk_img)

    sitk.WriteImage(imgData_Processed, outputPath)
    print(f'【Normalization processed successfully】{imagePath}')

In [6]:
# # 处理一个看看
# imagePath = r't2flair.nii.gz'
# N4BiasFieldCorrection(imagePath,True)

# imagePath = r"t2flair.nii.gz"
# signalStrengthNormalization(imagePath)

# 正式处理

In [23]:
import SimpleITK as sitk
from my_MRI_tool_kit import  DirectoryReader_dict
import numpy as np
import os

In [24]:
# 找到对应文件夹里所有文件
root_path = r"C:\Users\Administrator\Desktop\20230808"
max_depth = 1
directory_reader = DirectoryReader_dict(root_path, max_depth)
directories = directory_reader.get_files_dict()
print(f'共找到{len(directories)}个文件夹')

directories

共找到1个文件夹


{'C:\\Users\\Administrator\\Desktop\\20230808': ['1.【配准】重采样后互相刚体配准.ipynb',
  '2.【配准】Syn配准.ipynb',
  '3.【剥颅骨】应用BrainMask.ipynb',
  '4.【N4bias】.ipynb',
  '5【特征提取】.ipynb',
  'ce.nii.gz',
  'my_MRI_tool_kit.py',
  't1.nii.gz',
  't2.nii.gz',
  '__seg.nii.gz',
  '________b_regd_rsmp_ADCm.nii.gz',
  '________ws_b_regd_rsmp_fDWI_2.nii.gz',
  '__________CDK1m_seg.nii.gz',
  '【Zscore】.ipynb',
  '图像重采样.ipynb']}

# 重采样

In [26]:
aim_directories = {}
for key, value in directories.items():
    pattern =['ce.nii.gz','t1.nii.gz','t2.nii.gz']
    collection = []
    for i in value:
        if i in pattern:
            collection.append(i)
    aim_directories.update({key:collection})
    
aim_directories

{'C:\\Users\\Administrator\\Desktop\\20230808': ['ce.nii.gz',
  't1.nii.gz',
  't2.nii.gz']}

In [32]:
for key, value in aim_directories.items():
    for i in value:
        full_path = os.path.join(key,i)
        print(f"{'#'*30}")
        print(full_path)
        
        rsmp_output = os.path.join(key,'_rsmp_'+i)
        Zscore_output = os.path.join(key,'__Zscore_rsmp'+i)
        
        resample_image_execute(full_path, rsmp_output)
        signalStrengthNormalization(rsmp_output, Zscore_output)
        

##############################
C:\Users\Administrator\Desktop\20230808\ce.nii.gz
原始图像的Spacing： (0.8125, 0.8125, 4.400000095367432)
原始图像的Size： (320, 320, 10)
经过resample之后图像的Spacing是： (1.0, 1.0, 1.0)
经过resample之后图像的Size是： (260, 260, 44)
【Normalization processed successfully】C:\Users\Administrator\Desktop\20230808\_rsmp_ce.nii.gz
##############################
C:\Users\Administrator\Desktop\20230808\t1.nii.gz
原始图像的Spacing： (0.8125, 0.8125, 4.400000095367432)
原始图像的Size： (320, 320, 10)
经过resample之后图像的Spacing是： (1.0, 1.0, 1.0)
经过resample之后图像的Size是： (260, 260, 44)
【Normalization processed successfully】C:\Users\Administrator\Desktop\20230808\_rsmp_t1.nii.gz
##############################
C:\Users\Administrator\Desktop\20230808\t2.nii.gz
原始图像的Spacing： (0.8125, 0.8125, 4.400000095367432)
原始图像的Size： (320, 320, 10)
经过resample之后图像的Spacing是： (1.0, 1.0, 1.0)
经过resample之后图像的Size是： (260, 260, 44)
【Normalization processed successfully】C:\Users\Administrator\Desktop\20230808\_rsmp_t2.nii.gz
